In [ ]:
# This takes periods as found by RW, NDL and combines them to find an established period

# Created 2022 May 22 by E.S.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_name = "/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/data/input_periods_all_stars.csv"

In [3]:
df = pd.read_csv(file_name)

In [4]:
# initialize

df["err_diff"] = np.nan
df["err_tot"] = np.nan

In [5]:
# find final periods: simple average across values

cols = ['T_KELT', 'T_TESS', 'T_other']

df["T_final"] = df[cols].mean(axis=1)

In [6]:
'''
1. case where both KELT-based and TESS-based periods are available:

	err_diff = abs(T_NDL - T_RW)

	err_tot**2 = err_RW**2  + err_diff**2


2. case where only TESS-based period available:

	err_diff_avg = avg[  err_diff  ]

	err_tot**2 = err_RW**2  + err_diff_avg**2


3. case where only KELT-based period available:

	err_diff_avg = avg[  err_diff  ]
	err_RW_avg = avg[  err_RW ]

	err_tot**2 = err_RW_avg**2  + err_diff_avg**2
    
4. case where neither TESS nor KELT periods are available:

    just take average of the others
'''

'\n1. case where both KELT-based and TESS-based periods are available:\n\n\terr_diff = abs(T_NDL - T_RW)\n\n\terr_tot**2 = err_RW**2  + err_diff**2\n\n\n2. case where only TESS-based period available:\n\n\terr_diff_avg = avg[  err_diff  ]\n\n\terr_tot**2 = err_RW**2  + err_diff_avg**2\n\n\n3. case where only KELT-based period available:\n\n\terr_diff_avg = avg[  err_diff  ]\n\terr_RW_avg = avg[  err_RW ]\n\n\terr_tot**2 = err_RW_avg**2  + err_diff_avg**2\n    \n4. case where neither TESS nor KELT periods are available:\n\n    just take average of the others\n'

In [7]:
# 1. case where both KELT-based and TESS-based periods are available:
for i in range(0,len(df)):
    #print(i)
    # check if there are periods from both KELT and TESS (but TESS has no error)
    T_TESS = df["T_TESS"].iloc[i]
    T_KELT = df["T_KELT"].iloc[i]
    err_T_TESS = df["err_T_TESS"].iloc[i]
    #print(T_TESS, T_KELT)

    if ~np.isnan(T_TESS) and ~np.isnan(T_KELT):
        #print('i')
        err_diff = np.abs(np.subtract(T_TESS,T_KELT))
        df["err_diff"].loc[i] = err_diff
        
        # propagating error for an average gives 0.5*sqrt(err_1**2 + err_2**2), but here 
        # we don't have the error from KELT; so coefficient of 1 (instead of 0.5) here may be overestimate
        df["err_tot"].loc[i] = np.sqrt(np.add(np.power(err_T_TESS,2.),np.power(err_diff,2.)))
        
        # for checking
        print("------")
        print("star", df["star"].iloc[i])
        print("err_diff", err_diff)
        print("err_T_TESS", err_T_TESS)
        print("err_tot", df["err_tot"].loc[i])

/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


------
star RR Leo
err_diff 7.922999999965263e-07
err_T_TESS 6.79821337e-07
err_tot 1.0439810056868674e-06
------
star TT Lyn
err_diff 2.157499999988488e-06
err_T_TESS 1.7812966e-06
err_tot 2.7978248385347296e-06
------
star TV Lyn
err_diff 3.3486000000082505e-06
err_T_TESS 3.7242012023e-07
err_tot 3.369246014467834e-06
------
star TW Lyn
err_diff 1.0119999999669282e-06
err_T_TESS 1.7750202e-06
err_tot 2.0432426949193046e-06
------
star RR Lyr
err_diff 1.239990000001523e-05
err_T_TESS 4.41447566e-06
err_tot 1.3162261027768372e-05
------
star TU UMa
err_diff 1.467389999998403e-05
err_T_TESS 2.58581005e-06
err_tot 1.4899991772622304e-05


In [8]:
for i in range(0,len(df)):
    # check if there are errors from both KELT and TESS
    T_TESS = df["T_TESS"].iloc[i]
    T_KELT = df["T_KELT"].iloc[i]
    err_T_TESS = df["err_T_TESS"].iloc[i]

    # 2. case where only TESS-based period available:
    if ~np.isnan(T_TESS) and np.isnan(T_KELT):
        err_diff_avg = np.nanmean(df["err_diff"])
        df["err_tot"].loc[i] = np.sqrt(np.add(np.power(err_T_TESS,2.),np.power(err_diff_avg,2.)))
        
        # for checking
        print("------")
        print("star", df["star"].iloc[i])
        print("err_diff_avg", err_diff_avg)
        print("err_T_TESS", err_T_TESS)
        print("err_tot", df["err_tot"].loc[i])
        
    # 3. case where only KELT-based period available:
    elif np.isnan(T_TESS) and ~np.isnan(T_KELT):
        err_TESS_avg = np.nanmean(df["err_T_TESS"])
        df["err_tot"].loc[i] = np.sqrt(np.add(np.power(err_TESS_avg,2.),np.power(err_diff_avg,2.)))
        
        # for checking
        print("------")
        print("star", df["star"].iloc[i])
        print("err_diff_avg", err_diff_avg)
        print("err_TESS_avg", err_TESS_avg)
        print("err_tot", df["err_tot"].loc[i])


# average total error so far
avg_err_tot_empirical = np.nanmean(df["err_tot"])

for i in range(0,len(df)):
    # 4. case where neither TESS nor KELT periods are available:
    T_TESS = df["T_TESS"].iloc[i]
    T_KELT = df["T_KELT"].iloc[i]
    if np.isnan(T_TESS) and np.isnan(T_KELT):
        df["err_tot"].loc[i] = avg_err_tot_empirical

------
star RW Ari
err_diff_avg 5.730699999993242e-06
err_T_TESS 1.491772962e-05
err_tot 1.5980600098411446e-05
------
star X Ari
err_diff_avg 5.730699999993242e-06
err_T_TESS 7.549117669999999e-06
err_tot 9.47787423874303e-06
------
star UY Cam
err_diff_avg 5.730699999993242e-06
err_T_TESS 4.3600000000000004e-07
err_tot 5.747261825419349e-06
------
star RR Cet
err_diff_avg 5.730699999993242e-06
err_T_TESS 3.3428131400000002e-06
err_tot 6.634404432869556e-06
------
star SV Eri
err_diff_avg 5.730699999993242e-06
err_T_TESS 1.74047143e-06
err_tot 5.98917050087646e-06
------
star V535 Mon
err_diff_avg 5.730699999993242e-06
err_T_TESS 1.5029437e-07
err_tot 5.73267048482435e-06
------
star V445 Oph
err_diff_avg 5.730699999993242e-06
err_TESS_avg 3.0788197074792306e-06
err_tot 6.505386482068921e-06
------
star AV Peg
err_diff_avg 5.730699999993242e-06
err_TESS_avg 3.0788197074792306e-06
err_tot 6.505386482068921e-06
------
star AR Per
err_diff_avg 5.730699999993242e-06
err_TESS_avg 3.0788197

In [19]:
output_file_name = "junk.csv"
df.reset_index(drop=True, inplace=True)
df.to_csv(output_file_name, index=False)
print("Wrote ",output_file_name)

Wrote  junk.csv
